In [1]:
import pandas as pd

Считывание датасета

In [2]:
df = pd.read_csv('train.csv')
print("1. Датасет загружен")
print(f"Размер датасета: {df.shape}")
print()

1. Датасет загружен
Размер датасета: (891, 12)



Основная информация о датасете

In [3]:
print("Информация о типах данных и пропусках:")
print(df.info())
print("\nОписательная статистика:")
print(df.describe(include='all'))
print("\nКоличество пропусков по столбцам:")
print(df.isnull().sum())
print()

Информация о типах данных и пропусках:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None

Описательная статистика:
        PassengerId    Survived      Pclass                 Name   Sex  \
count    891.000000  891.000000  891.000000                  891   891   
unique          NaN       

Процент выживаемости у каждого класса пассажиров

In [6]:
survival_by_class = df.groupby('Pclass')['Survived'].mean() * 100
print(survival_by_class)
print()

Pclass
1    62.962963
2    47.282609
3    24.236253
Name: Survived, dtype: float64



Самое популярное мужское и самое популярное женское имя на корабле

In [26]:
def extract_first_name(name):
    """Функция извлечения имени"""
    if '(' in name and ')' in name:
        bracket_content = name.split('(')[1].split(')')[0]
        clean_name = bracket_content.replace('Mrs. ', '').replace('Miss. ', '')
        first_name = clean_name.split(' ')[0]
        if first_name:
            return first_name


    parts = name.replace(',', '').split(' ')


    titles = ['Mrs.', 'Miss.', 'Mr.', 'Master.']
    for i, part in enumerate(parts):
        if part in titles and i + 1 < len(parts):
            return parts[i + 1]  # Слово после титула - это имя

    if len(parts) > 1:
        return parts[1]  # Второе слово обычно имя

    return parts[0]


In [27]:
male_names = df[df['Sex'] == 'male']['FirstName']
female_names = df[df['Sex'] == 'female']['FirstName']

print(f"Самое популярное мужское имя: {male_names.mode().iloc[0]}")
print(f"Самое популярное женское имя: {female_names.mode().iloc[0]}")
print()

Самое популярное мужское имя: William
Самое популярное женское имя: Anna



 Самое популярное мужское и самое популярное женское имя на корабле в каждом классе

In [28]:
for pclass in sorted(df['Pclass'].unique()):
    class_data = df[df['Pclass'] == pclass]
    male_class = class_data[class_data['Sex'] == 'male']['FirstName']
    female_class = class_data[class_data['Sex'] == 'female']['FirstName']

    print(f"Класс {pclass}:")
    print(f"  Мужское: {male_class.mode().iloc[0] if not male_class.empty else 'Нет данных'}")
    print(f"  Женское: {female_class.mode().iloc[0] if not female_class.empty else 'Нет данных'}")
print()

Класс 1:
  Мужское: William
  Женское: Elizabeth
Класс 2:
  Мужское: William
  Женское: Elizabeth
Класс 3:
  Мужское: William
  Женское: Anna



Пассажиры, возраст которых больше 44 лет

In [16]:
passengers_over_44 = df[df['Age'] > 44]
print(f"Количество пассажиров старше 44 лет: {len(passengers_over_44)}")
print(passengers_over_44[['Name', 'Age', 'Sex', 'Pclass']].head())
print()

Количество пассажиров старше 44 лет: 115
                                        Name   Age     Sex  Pclass
6                    McCarthy, Mr. Timothy J  54.0    male       1
11                  Bonnell, Miss. Elizabeth  58.0  female       1
15          Hewlett, Mrs. (Mary D Kingcome)   55.0  female       2
33                     Wheadon, Mr. Edward H  66.0    male       2
52  Harper, Mrs. Henry Sleeper (Myna Haxtun)  49.0  female       1



Пассажиры, возраст которых меньше 44 лет и которые мужского пола

In [12]:
young_male = df[(df['Age'] < 44) & (df['Sex'] == 'male')]
print(f"Количество пассажиров младше 44 лет и мужского пола: {len(young_male)}")
print(young_male[['Name', 'Age', 'Sex', 'Pclass']].head())
print()

Количество пассажиров младше 44 лет и мужского пола: 368
                              Name   Age   Sex  Pclass
0          Braund, Mr. Owen Harris  22.0  male       3
4         Allen, Mr. William Henry  35.0  male       3
7   Palsson, Master. Gosta Leonard   2.0  male       3
12  Saundercock, Mr. William Henry  20.0  male       3
13     Andersson, Mr. Anders Johan  39.0  male       3



 Количество n-местных кабин (в которых было 2, 3, 4, ... человека)

In [32]:
cabins_df = df.dropna(subset=["Cabin"]).copy()
cabins_df["CabinT"] = cabins_df["Cabin"].str.split()
exploded = cabins_df.explode("CabinT")

occupancy_per_cabin = exploded["CabinT"].value_counts()

n_place_counts = occupancy_per_cabin.value_counts().sort_index()

print("Количество n-местных кают:")
for n, cnt in n_place_counts.items():
    print(f"  {n}-местных кают: {cnt}")


Количество n-местных кают:
  1-местных кают: 104
  2-местных кают: 44
  3-местных кают: 6
  4-местных кают: 7
